In [8]:
import pandas as pd
import numpy as np
import xml.etree
import matplotlib.pyplot as plt
import seaborn as sns
from implementations import *
import csv
import gzip
import xml.etree.ElementTree as ET

In [2]:
character_md = load_character_md()
tvtropes_clusters = load_tvtropes_clusters()
movie_md = load_movie_md()
name_clusters = load_name_clusters()
plot_summaries = load_plot_summaries()

In [56]:
movie_ids = character_md.drop_duplicates('Wikipedia movie ID')['Wikipedia movie ID']

## we pre-allocate the list so loops goes faster
SIZE = 10000000
dependency = [None] * SIZE
governor = [None] * SIZE
dependent = [None] * SIZE
actor_name = [None] * SIZE
actor_id = [None] * SIZE
actor_gender = [None] * SIZE
char_name = [None] * SIZE
action = [None] * SIZE

## start loop over each movie

i = 0
j = 0
for movie_id in movie_ids:
    
    ## to have an idea of how much mivies we processed
    
    if i%50 == 0:
        print(i)
    i+=1
    if i == 1000:
        break
        
    ## we open the file in relation to this movie_id
    
    PATH = './MovieSummaries/corenlp_plot_summaries/' + str(movie_id) + '.xml.gz'
    
    try: 
        UNZIP_PATH = gzip.open(PATH, 'r')
        tree = ET.parse(UNZIP_PATH)
        root = tree.getroot()
    except:
        ## Some movies in the data base don't have a resume that have 
        ## been processed by the stanford coreNLP program.
        ## We just skip for these movies
        continue
        
    characters = character_md[character_md['Wikipedia movie ID'] == movie_id].dropna(subset=['Character name'])
    
    ## start loop over all dependencies
    for child in root.iter('dep'):
        
        typ = child.attrib['type']
        gov = child[0].text
        dep = child[1].text
        
        ## start loop over all characters in the movie
        for idx, prsn in characters.iterrows():
    
            chn = prsn['Character name']
            actorname = prsn['Actor name']
            actorid = prsn['Freebase actor ID']
            actorgender = prsn['Actor gender']
            
            ## often, a character is called by only a part of its name, 
            ## we have to take it into account.
            for chn_ in chn.split():
                if (chn_ == gov) | (chn_ == dep):
                    actor_name[j] = actorname
                    actor_id[j] = actorid
                    actor_gender[j] = actorgender
                    char_name[j] = chn
                    dependency[j] = typ
                    governor[j] = gov
                    dependent[j] = dep
                    if chn_ == gov:
                        action[j] = dep
                    else:
                        action[j] = gov
                    j+=1
                    
                    
actor_words = pd.DataFrame.from_dict({'actor name' : actor_name[:j], 'actor id' : actor_id[:j], 'actor gender' : actor_gender[:j], 'character name' : char_name[:j], 'dependency':dependency[:j], 'governor':governor[:j], 'dependent':dependent[:j], 'action' : action[:j]})

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950


In [57]:
act_df = Actors_words.drop_duplicates(subset=['character name', 'action'])

In [61]:
act_df[act_df['actor gender']=='M'].groupby('action').count().sort_values('actor name', ascending= False).head(30)

,actor name,actor id,actor gender,character name,dependency,governor,dependent
action,,,,,,,
's,241,241,241,241,241,241,241
and,226,226,226,226,226,226,226
with,115,115,115,115,115,115,115
of,113,113,113,113,113,113,113
by,95,95,95,95,95,95,95
to,93,93,93,93,93,93,93
the,67,67,67,67,67,67,67
tells,66,66,66,66,66,66,66
for,56,56,56,56,56,56,56
